## SLIQ

In [2]:
import pandas as pd
import numpy as np
from ipynb.fs.full.algorithm_u import algorithm_u
from collections import Counter
eps = np.finfo(float).eps
from numpy import log2 as log

In [4]:
df = pd.read_csv('sliq.csv')
df

,rid,age,salary,marital,car
0,1,30,60,single,sports
1,2,25,20,single,mini
2,3,40,80,married,van
3,4,45,100,single,luxury
4,5,60,150,married,luxury
5,6,35,120,single,sports
6,7,50,70,married,van
7,8,55,90,single,sports
8,9,65,30,married,mini
9,10,70,200,single,luxury


In [77]:
def entropy(df):
    entropy_node = 0
    lastcol = df.iloc[:,-1]
    values = lastcol.unique() 
    for value in values:
        fraction = lastcol.value_counts()[value]/len(lastcol)  
        entropy_node += -fraction*np.log2(fraction)
    return entropy_node

In [78]:
entropy(df)

1.9709505944546688

In [75]:
def attribute_entropy(df,i):
    attribute = i
    lastcol = df.iloc[:,-1]
    target_variables = lastcol.unique()
    variables = df[attribute].unique() 
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            numerator = len(df[attribute][df[attribute]==variable][lastcol ==target_variable])
            denominator = len(df[attribute][df[attribute]==variable])
            fraction = numerator/(denominator+eps)  #pi
            entropy_each_feature += -fraction*log(fraction+eps)
        fraction2 = denominator/len(df)
        entropy_attribute += -fraction2*entropy_each_feature 
    return entropy_attribute

In [76]:
attribute_list = df.iloc[:, 1:4]
attribute_list

for i in attribute_list:
    print(i, attribute_entropy(df,i))

age 0.0
salary 0.0
marital -1.4754887502163458


In [14]:
def info_gain(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(entropy(df)-attribute_entropy(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [15]:
info_gain(df)

'marital'

In [251]:
def attribute_lists():
    age_attr_list = df[['rid', 'age']].sort_values(by=['age'])
    salary_attr_list = df[['rid', 'salary']].sort_values(by=['salary'])
    marital_attr_list = df[['rid', 'marital']].sort_values(by=['marital'])
    class_attr_list = df[['rid', 'car']].sort_values(by=['car'])

In [260]:
class_attribute_list = df['car']
unique_class_values, unique_class_count = np.unique(class_attribute_list, return_counts=True)
print(unique_class_values, unique_class_count)

['luxury' 'mini' 'sports' 'van'] [3 2 3 2]


In [261]:
dat = pd.DataFrame(unique_class_values)
dat.columns = ["class"]

right = unique_class_count
dat['right'] = right 
print(dat)

    class  right
0  luxury      3
1    mini      2
2  sports      3
3     van      2


In [262]:
def histogram():
    left = []
    right = []
    for j, jrow in age_attr_list.iterrows():
        if jrow['age'] <= 30:
            for index, row in class_attr_list.iterrows():
                if j == index:
                    for k, krow in dat.iterrows():
                        if row['car'] == krow['class']:
                            dat.loc[dat.index[k], 'Left'] = 1
                            dat.at[k,'right']= krow['right'] - 1
    return dat

In [263]:
histogram()

,class,right,Left
0,luxury,3,NaN
1,mini,1,1.0
2,sports,2,1.0
3,van,2,NaN


In [264]:
def histo():
    left = []
    right = []
    for j, jrow in salary_attr_list.iterrows():
        if jrow['salary'] <= 60:
            for index, row in class_attr_list.iterrows():
                if j == index:
                    for k, krow in dat.iterrows():
                        if row['car'] == krow['class']:
                            dat.loc[dat.index[k], 'Left'] = 1
                            dat.at[k,'right']= krow['right'] - 1
    
    return dat

In [265]:
histo()

,class,right,Left
0,luxury,3,NaN
1,mini,-1,1.0
2,sports,1,1.0
3,van,2,NaN


In [246]:
def split_atribute(leaflist):
    print("leaflist: ", leaflist)
    last = leaflist[0][0]
    e_index = -1
    index = -1
    best_index = 0
    best_gini = 2
    best_entropy = 2
    best_entropy_index = 0
    for value in leaflist:  # for every unique value
        index += 1
        e_index += 1
        if value[0] != last:
            last = value[0]
            right = leaflist[index:]
            left = leaflist[0:index]
            gini = calculate_gini_split(left, right, dictrid)
            entrop = calculate_overall_entropy(left, right)
            if best_gini > gini:
                best_gini = gini
                best_index = index

    print("\n Best gini: ", best_gini)
    return best_gini, leaflist[best_index][0]

In [273]:
def choose_split(leafset, dataset, depth):
    best = 2
    best_split = 0
    best_list = []
    column_index = -1
    best_column_index = 0
    listij = []
    for list in dataset:  
        column_index += 1
        
        for item in list:
            for j in leaf:
                if item == j:
                    listij.append([item,(item-1)])
        
    leaf_list = listij
    for list in dataset:  
        column_index += 1
        listij2 = []
        for item in list:
            for j in leaf:
                if item == j:
                    listij2.append([item,(item)])
        
        
        temp, split = split_atribute(leaf_list)

        if best > temp:  
            best = temp
            best_split = split
            best_column_index = column_index
            best_list = leaf_list
        if temp == 0:  
            break
            
    print("best split: ", best_split)
    print("best list: ", best_list)
    print("best column index: ", best_column_index)

In [306]:
dataset = df.values.tolist()
print("dataset", dataset)

ridl = df.iloc[:, 0].tolist()
print("\n ridl", ridl)

leaf = {item for item in range(0, len(ridl))}
print("\n leaf: ", leaf)

 # create a dictionary for the labels
length = len(df)
num_cols = len(df.columns)
dictrid = dict()
for index in range(length):
    dictrid.update({index: df[df.columns[num_cols - 1]].iloc[index]})

choose_split(leaf, dataset, 1)


dataset [[1, 30, 60, 'single', 'sports'], [2, 25, 20, 'single', 'mini'], [3, 40, 80, 'married', 'van'], [4, 45, 100, 'single', 'luxury'], [5, 60, 150, 'married', 'luxury'], [6, 35, 120, 'single', 'sports'], [7, 50, 70, 'married', 'van'], [8, 55, 90, 'single', 'sports'], [9, 65, 30, 'married', 'mini'], [10, 70, 200, 'single', 'luxury']]

 ridl [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

 leaf:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
leaflist:  [[1, 0], [2, 1], [3, 2], [4, 3], [5, 4], [6, 5], [7, 6], [8, 7], [9, 8]]

 Best gini:  0.0
best split:  2
best list:  [[1, 0], [2, 1], [3, 2], [4, 3], [5, 4], [6, 5], [7, 6], [8, 7], [9, 8]]
best column index:  10


In [302]:
def calculate_gini_split(leaflist_left, leaflist_right, classlist):
    n_left = len(leaflist_left)
    n_right = len(leaflist_right)

    if n_left == 0 or n_right == 0:  # check for empty lists
        return 2

    n = n_left + n_right
    positive_left = 0.0
    positive_right = 0.0

    for item in leaflist_left:
        if classlist[item[1]] == 1:
            positive_left += 1

    for item in leaflist_right:
        if classlist[item[1]] == 1:
            positive_right += 1

    neg_a = n_left - positive_left
    neg_b = n_right - positive_right
    gini_a = 1 - float((positive_left * positive_left + neg_a * neg_a) / (n_left * n_left))
    gini_b = 1 - float((positive_right * positive_right + neg_b * neg_b) / (n_right * n_right))
    return (n_left / n) * gini_a + (n_right / n) * gini_b